In [1]:
import abtem
import ase
import dask
from IPython.display import Image, display 

dask.config.set({"array.svg.size": 90});

(walkthrough:parallelization)=
# Parallelization

The computational cost of running multislice simulations can become high depending on the size of the specimen, the number of probe positions, phonon images, and many other factors. This cost can be mitigated using parallelism. Many of the computations in *ab*TEM are [embarrasingly parallel](https://en.wikipedia.org/wiki/Embarrassingly_parallel): for example, every probe position is independent, thus each CPU core may calculate a batch of positions independently, only requiring communication after finishing a run of the multislice algorithm. 

*ab*TEM is parallelized using [Dask](https://www.dask.org/){cite}`dask`, which is a flexible library for parallel computing in Python. Dask allows scaling from a single laptop to hundreds of nodes at high-performance computing (HPC) facilities with minimal changes to the code. 

In this walkthrough, we introduce how *ab*TEM uses Dask. Althouhg this is not required knowledge for running simulations on a single machine, it may still help you optimize your simulations. If you are already an experienced Dask user, most of what you already know can be applied to using *ab*TEM. If you are new to Dask you may benefit from watching [this introduction](https://www.youtube.com/watch?v=nnndxbr_Xq4) before continuing.

We note that Dask is used in several other libraries for the analysis of electron microscopy data, for example, [hyperspy](https://hyperspy.org/), [libertem](https://libertem.github.io/LiberTEM/) and [py4DSTEM](https://py4dstem.readthedocs.io/en/latest/), and thus we think that you may benefit from knowing this library more generally.

## Task graphs

Simulating TEM experiments requires executing multiple *tasks* where each one may depend on the output of previous tasks. In Dask this is represented as a [*task graph*](https://docs.dask.org/en/stable/graphs.html), where each task is a *node*, with *edges* between nodes if it is dependent on another task. The simulation result is obtained by executing each task (node) in the graph with a Dask *scheduler* on a single machine or a cluster.

In [2]:
display(
    Image(url="https://docs.dask.org/en/stable/_images/dask-overview.svg", width=600)
)

To illustrate this in practice, below we create the task graph for running a plane wave multislice simulation of gold in the $\left<100\right>$ zone axis with 4 frozen phonon configurations (please refer to previous walkthroughs for those details).

In [3]:
atoms = ase.build.bulk("Au", cubic=True) * (5, 5, 2)

frozen_phonons = abtem.FrozenPhonons(
    atoms, num_configs=4, sigmas=0.1, ensemble_mean=False
)

potential = abtem.Potential(frozen_phonons, gpts=512, slice_thickness=2)

probe = abtem.PlaneWave(energy=200e3)

exit_waves = probe.multislice(potential)

The result is an ensemble of $4$ wave functions of shape $512\times512$, which may be represented as a 3D array, where the first dimension represents the phonon ensemble and the last two dimensions represents the 2D wave functions. 

As we have not executed the task graph yet, the wave functions are represented as a [Dask Array](https://docs.dask.org/en/stable/array.html). We can think of the *Dask array* as being composed of many smaller NumPy arrays, called *chunks*, and operations may be applied to each chunk rather than the full array. This enables: 

1. Parallelism over the chunks;
2. Representing a larger-than-memory array as many smaller arrays, each of which fits in memory.

The Dask array representation `__repr__` shows how the chunks are laid out.

In [4]:
exit_waves.array

dask.array<_extract_measurement, shape=(4, 512, 512), dtype=complex64, chunksize=(1, 512, 512), chunktype=numpy.ndarray>

We see that this Dask array has the shape `(4, 512, 512)` requiring 8 MB of memory, and is composed of chunks with shape `(1, 512, 512)` requiring 2 MB each.

```{important}
The Dask array just represents a task graph, not the data itself, so memory is consumed only if it is computed!
```

Each chunk of the Dask array created above represents a wave function for a frozen phonon configuration. This reflects the fact that, in the multislice algorithm, each frozen phonon configuration is independent and may be calculated in parallel. On the other hand, we should not have chunks across wave functions, because each part of a wave function is affected by every other part.

We can visualize the task graph using Dask's [`visualize`](https://docs.dask.org/en/stable/graphviz.html) method. We see that the task graph consists of 4 fully independent branches, one for each frozen phonon.

````{note}
Drawing Dask task graphs with the cytoscape engine requires the `ipycytoscape` python library. To reproduce the result below you need to:
```
  python -m pip install ipycytoscape
```
and restart jupyter.
````

In [5]:
exit_waves.array.visualize(engine="cytoscape")

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

We usually want to take the mean across the frozen phonon dimension, and thus end up with an image represented as a single chunk.

In [6]:
hrtem_image = exit_waves.intensity().mean(0)

hrtem_image.array

dask.array<mean_agg-aggregate, shape=(512, 512), dtype=float32, chunksize=(512, 512), chunktype=numpy.ndarray>

Taking the mean across frozen phonon chunks requires communicating the exit wave function intensity between the final nodes. Showing the task graph again we see how the branches are merged when the result have to be communicated between workers.

In [7]:
hrtem_image.array.visualize(engine="cytoscape")

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

## Chunks
To futher explore the role of chunks in abTEM, we create the task graph for running a STEM simulation with the same specimen, gold in the $\left<100\right>$ zone axis with 4 frozen phonons. We do not immediately apply a detector, so we obtain an ensemble of exit wave functions.

In [8]:
probe = abtem.Probe(energy=200e3, semiangle_cutoff=20)

scan = abtem.GridScan.from_fractional_coordinates(
    potential,
    start=(0, 0),
    end=(1 / 5, 1 / 5),
)

frozen_phonons = abtem.FrozenPhonons(
    atoms, num_configs=4, sigmas=0.1, ensemble_mean=True
)

potential = abtem.Potential(frozen_phonons, gpts=512, slice_thickness=2)

exit_waves_stem = probe.multislice(potential, scan=scan)

exit_waves_stem.axes_metadata

type               label                 coordinates
-----------------  --------------------  -------------------
FrozenPhononsAxis  Frozen phonons        -
ScanAxis           $x \ [\mathrm{\AA}]$  0.00 0.29 ... 3.79
ScanAxis           $y \ [\mathrm{\AA}]$  0.00 0.29 ... 3.79
RealSpaceAxis      $x \ [\mathrm{\AA}]$  0.00 0.04 ... 20.36
RealSpaceAxis      $y \ [\mathrm{\AA}]$  0.00 0.04 ... 20.36

The wave functions are now represented as a 5D Dask array, a 3D ensemble of 2D wave functions, which is composed of one frozen phonon dimension and two scan dimensions, one for each of the $x$ and $y$ direction. 

The full array is of shape `(4, 15, 15, 512, 512)` requiring 1.76 GB of memory, which is cut into chunks of shape `(1, 8, 7, 512, 512)` of 112 MB. Hence, there are a total of $4 \times 2 \times 3 = 24$ chunks.

In [9]:
exit_waves_stem.array

dask.array<_extract_measurement, shape=(4, 14, 14, 512, 512), dtype=complex64, chunksize=(1, 8, 7, 512, 512), chunktype=numpy.ndarray>

Notice that we do not make a chunk for every probe position, but each chunk of the scan dimension instead represents a batch of wave functions. This is done partly to limit [the overhead](https://docs.dask.org/en/stable/best-practices.html#avoid-very-large-graphs) that every chunk comes with, but more importantly, larger batches enables efficient thread parallelization within each run of the multislice algorithm.

We can change how many wave functions each batch should have using a keyword. Below we set `max_batch=4`, resulting in a total number of $4 \times 8 \times 8 = 256$ chunks.

In [10]:
exit_waves_stem = probe.multislice(potential, scan=scan, max_batch=4)

exit_waves_stem.array

dask.array<_extract_measurement, shape=(4, 14, 14, 512, 512), dtype=complex64, chunksize=(1, 2, 2, 512, 512), chunktype=numpy.ndarray>

The default value of `max_batch` is `"auto"`: with this setting the number of wave functions in each batch is determined such that the batch represents approximately `128 MB` of memory, although this number may be changed through the *ab*TEM [configuration](../../reference/config.md).

Before computing this task graph, we apply a HAADF detector and calculate the ensemble mean, which reduces the total size of the output to just 900 B. We note that the 1.76 GB ensemble of wave functions never needs to be in memory simulateneously, as each chunk of exit wave functions are reduced immediately after completing the multislice algorithm.

In [11]:
detector = abtem.AnnularDetector(inner=65, outer=200)

haadf_images = detector.detect(exit_waves_stem).reduce_ensemble()

haadf_images.array

dask.array<mean_agg-aggregate, shape=(14, 14), dtype=float32, chunksize=(2, 2), chunktype=numpy.ndarray>

## Schedulers

After generating a task graph, it needs to be executed on (parallel) hardware. This is the job of a [task scheduler](https://docs.dask.org/en/stable/scheduler-overview.html). Dask provides several *task schedulers*: each of which will compute a task graph and give the same result, but with different performance characteristics.

Every time you call the `compute` method a Dask scheduler is used. We adopt the default Dask scheduler configuration and every keyword argument used with the `compute` method in *ab*TEM is forwarded to the Dask `compute` function. 

### Local scheduler

The default scheduler is the [`ThreadPoolExecutor`](https://docs.dask.org/en/stable/scheduling.html#local-threads). As an example of a keyword argument, the threaded scheduler takes `num_workers`, which sets the number of threads to use (by default, up to the number of computing cores). Let's run compute with 8 workers.

In [12]:
haadf_images = detector.detect(exit_waves_stem).reduce_ensemble()

haadf_images.compute(
    scheduler="threads", num_workers=8
)

[########################################] | 100% Completed | 15.77 s


We can change the scheduler to using the `ProcessPoolExecutor` as below.

```python
haadf_images = detector.detect(exit_waves_stem).compute(scheduler="processes", num_workers=4)
```

Using `abtem.config.set` the scheduler can be set either as a context manager or globally.
```python
# As a context manager
with abtem.config.set(scheduler="processes"):
    haadf_images.compute()

# Set globally
abtem.config.set(scheduler="processes")
haadf_images.compute()
```

### Profiling

To improve performance, we have to be able to profile it. Profiling parallel code can be challenging, but Dask provides functionality to aid in this and inspecting execution. The diagnostic tools are however quite different depending on whether you use a local or distributed scheduler.

Dask allows local diagnostics by adding callbacks that collect information about your code execution. You can use the profilers as a context manager as described in the [Dask documentation](https://docs.dask.org/en/stable/diagnostics-local.html). For convenience, the *ab*TEM `compute` methods also implement keywords for adding Dask profilers.

Below we use the [`Profiler`](https://docs.dask.org/en/stable/diagnostics-local.html#profiler) to monitor task execution by setting `profiler=True` and a [`ResourceProfiler`](https://docs.dask.org/en/stable/diagnostics-local.html#resourceprofiler) to monitor the CPU usage and memory consumption by setting `resource_profiler=True`. We rerun the simulation above with these profilers. 

In [13]:
haadf_images = detector.detect(exit_waves_stem).reduce_ensemble()

haadf_images, profilers = haadf_images.compute(profiler=True, resource_profiler=True)

[########################################] | 100% Completed | 14.59 s


To display the results Dask uses the [Bokeh plotting library](https://bokeh.org/) (which is installed together with Dask). To display the plots in a Jupyter notebook we need to run the commands below.

In [14]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

We first show the result from the `Profiler` object: This shows the execution time for each task as a rectangle, organized along the vertical axis by worker (in this case threads), and where white space represents times the worker was idle. The task types are grouped by color and, by hovering over each task, one can see the key and task that each block represents. For this calculation there is only one significant task shown in yellow; this encompasses building the wave function, running the multislice algorithm (calculating the potential on demand), and calculating and integrating the diffraction patterns. 

In [15]:
profilers[0].visualize();

The result from the `ResourceProfiler` is shown below: This shows two lines, one for total CPU percentage used by all the workers, and one for total memory usage. The CPU usage is scaled so each worker contributes up to $100 \ \%$, i.e. two fully utilized workers use $200 \ \%$.

In [16]:
profilers[1].visualize();

We ran the calculation on a single computer with an 4-core CPU with 2 threads per core for 8 threads total. We see that our peak CPU usage was $\sim 600 \%$, this is a fairly typical usage statistic for a single machine, overhead and background processes limited us from reaching the $800 \ \%$ corresponding to using every available thread maximally.

We also see that the total memory use reached around 800 MB. If you are running the calculation on a system with more threads your memory consumption may be larger – it may even exceed the total memory cost of all the wave functions, as every parallel run of the multislice algorithm requires a significant overhead for intermediate results (such as potential slices and Fresnel propagators). If your calculation runs out of memory you can lower the number of workers, thus trading away computational speed for lower memory consumption.

We note that the overhead in both CPU usage and memory diminishes for larger simulation with more powerful hardware.

### The (locally) distributed scheduler
The Dask distributed scheduler is necessary for running your simulations on a cluster, but it also runs [locally on a personal machine](https://docs.dask.org/en/stable/scheduling.html#dask-distributed-local). You can find details in the Dask documentation, but we demonstrate the basics below.

You can use the Dask distributed scheduler by just initializing a [Dask Client](https://distributed.dask.org/en/stable/client.html). The `Client` takes keyword arguments such as `n_workers` (note that this is different from `num_workers` above!)

In [17]:
from dask.distributed import Client

client = Client(n_workers=6)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 18,Total memory: 31.93 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:65243,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 18
Started: Just now,Total memory: 31.93 GiB
Comm: tcp://127.0.0.1:65280,Total threads: 3
Dashboard: http://127.0.0.1:65283/status,Memory: 5.32 GiB
Nanny: tcp://127.0.0.1:65246,


After intializing the client object, any *ab*TEM computation will use the Dask distributed scheduler.

In [18]:
haadf_images = detector.detect(exit_waves_stem).reduce_ensemble()
haadf_images.compute() #compute()

A benefit of using the distributed scheduler on a single machine is the live diagnostic dashboard. You can access this through the Dashboard link shown in the `__repr__` for the `Client` above (for details you can watch [this](https://www.youtube.com/watch?v=N_GqzcuGLCY) video walkthrough). If you are using Jupyter Lab, the [Dask labextension](https://github.com/dask/dask-labextension) provides the same information as a panel inside the Jupyter Lab editor.

We can get back to the local scheduler by closing the `Client`.

In [19]:
client.close()

### Running *ab*TEM on HPC clusters

Dask (and thus *ab*TEM) has robust tools for deployment on high-performance compute clusters. We recommend consulting your HPC provider on how to deploy Dask applications on your available cluster. For general advice on deployment, please see the [Dask documentation](https://docs.dask.org/en/stable/deploying.html).

#### Submitting job scripts 

As an overview, Dask provides a number of different cluster managers, so you can use distributed Dask on a range of platforms. These cluster managers deploy a scheduler and the necessary workers as determined by communicating with the resource manager. All cluster managers follow the same interface but have platform-specific configuration options.

As an example, for deployment using [SLURM](https://slurm.schedmd.com/documentation.html), your script might look something like:

```python
from dask_jobqueue import SLURMCluster
from dask.distributed import Client

cluster = SLURMCluster(
    queue="regular",
    account="myaccount",
    cores=32,
    memory="128 GB"
)

client = Client(cluster)

# Your abTEM code goes here

```

Dask also supports deployment from within an existing MPI environment, such as one created with the common MPI command-line launcher `mpirun`, see [here](http://mpi.dask.org/en/latest/) for more information. You can turn your batch Python script into an MPI executable with the `dask_mpi.initialize` function. 

```python
from dask_mpi import initialize
initialize()

from dask.distributed import Client
client = Client()  # Connect this local process to remote workers

# Your abTEM code goes here
```

This makes your Python script launchable directly with `mpirun`:
```
mpirun -np 4 python my_client_script.py
```

(walkthrough:parallelization:gpu)=
## Using GPUs

Almost every part of *ab*TEM can be accelerated using a GPU through the [CuPy](https://cupy.dev/) library. We have only tested CUDA-compatible GPUs, but any graphics card compatible with CuPy should work.

If you have a compatible GPU and a working installation of CuPy, you can accelerate your image simulations by simply changing the configs at the top of your document as below:
```python
abtem.config.set({"device": "gpu"});
```
On a single GPU, you may also want to limit the number of workers to one, `num_workers=1`. As described above, this may be done through the Dask config:
```python
dask.config.set({"num_workers": 1});
```

Note that Dask does not manage GPU threads. This makes the choice of batch sizes (i.e. propagating multiple wave functions in a single batch) extremely important in order to fully utilize your GPU. The default batch size in GPU calculations in abTEM is $512\ \mathrm{MB}$, which is four times larger than the CPU batch size, but if your GPU has $8\ \mathrm{GB}$ or more memory, you will likely be able to increase this number to $2048\ \mathrm{MB}$.

```python
abtem.config.set({"dask.chunk-size-gpu" : "512 MB"})
```

```{note}
The batch size only determines the maximum number of plane waves in a batch, so you need to leave room in the memory for any intermediate overhead. 
```

Finally, *ab*TEM by default sets the [FFT plan cache](https://docs.cupy.dev/en/stable/user_guide/fft.html#fft-plan-cache) size of `cupy` to zero, as we find that in most cases the increased memory consumption of the plans are not worth the small speedup they provide. You can however also change this through the *ab*TEM config.

```python
abtem.config.set({"cupy.fft-cache-size" : "1024 MB"})
```

### Multiple GPUs 
The above is enough for running *ab*TEM on a single GPU; if you are using an NVidia GPU, you may want to install `dask_cuda` (this is currently only supported on Linux). However, `dask_cuda` is currently necessary for multi-GPU calculations with *ab*TEM.

```python
from dask_cuda import LocalCUDACluster
from dask.distributed import Client

cluster = LocalCUDACluster()
client = Client(cluster)
```